<div style="text-align:center">
    <img src="imgs/cafe.png" alt="alt text" title="MSDS CAFE" width="500"/>
</div>

In [1]:
# Import libraries
import pandas as pd
from sqlalchemy import create_engine, text
from IPython.display import display

In [2]:
# Global vars
engine = create_engine('sqlite:///:memory:')

Things to note about the data:
- Menu has a new item that hasn't been sold yet
- Sales has a new item sold that isn't in the menu table yet

In [3]:
# Create the tables
files_and_tables = {
    'data/menu.csv': 'menu_table',
    'data/sales.csv': 'sales_table',
    'data/employees.csv': 'employees_table'
}

for filepath, tablename in files_and_tables.items():
    df = pd.read_csv(filepath)
    df.to_sql(tablename, engine, index=False)

# Fn to display results from query
def query_to_result(query):
    wrapped_query = text(query)
    result = pd.read_sql_query(wrapped_query, engine)
    display(result)

## TABLES

![alt text](imgs/ER_plot.png "ER diagram")

In [4]:
query = \
'SELECT * FROM menu_table'
query_to_result(query)

,item_id,item_name,price
0,1,Linear Regresso Latte,3.0
1,2,Croissant,2.5
2,3,Smoothie,4.0
3,4,None,2.0
4,5,Outlier Omelette,3.5


In [5]:
query = \
'SELECT * FROM sales_table'
query_to_result(query)

,sale_id,item_id,sale_date
0,1,1,2023-10-01
1,2,2,2023-10-01
2,3,3,2023-10-02
3,4,1,2023-10-03
4,5,4,2023-10-04
5,6,6,2023-10-05


In [6]:
query = \
'SELECT * FROM employees_table'
query_to_result(query)

,employee_id,employee_name,manager_id
0,1,Jiaxuan,5.0
1,2,Ireri,5.0
2,3,Rithvik,5.0
3,4,Bassim,5.0
4,5,Robert,NaN


## RELATIONAL JOINS

### INNER JOIN

In [7]:
query = \
'''
SELECT m.item_name, s.sale_date
FROM sales_table s
INNER JOIN menu_table m ON s.item_id = m.item_id;
'''
query_to_result(query)

,item_name,sale_date
0,Linear Regresso Latte,2023-10-01
1,Croissant,2023-10-01
2,Smoothie,2023-10-02
3,Linear Regresso Latte,2023-10-03
4,None,2023-10-04


### LEFT JOIN

In [8]:
query = \
'''
SELECT m.item_name, s.sale_date
FROM menu_table m
LEFT JOIN sales_table s ON m.item_id = s.item_id;
'''
query_to_result(query)


,item_name,sale_date
0,Linear Regresso Latte,2023-10-01
1,Linear Regresso Latte,2023-10-03
2,Croissant,2023-10-01
3,Smoothie,2023-10-02
4,None,2023-10-04
5,Outlier Omelette,None


### RIGHT JOIN

In [9]:
query = \
'''
SELECT m.item_name, s.sale_date
FROM menu_table m
RIGHT JOIN sales_table s ON m.item_id = s.item_id;
'''
query_to_result(query)


,item_name,sale_date
0,Linear Regresso Latte,2023-10-01
1,Linear Regresso Latte,2023-10-03
2,Croissant,2023-10-01
3,Smoothie,2023-10-02
4,None,2023-10-04
5,None,2023-10-05


### OUTER JOIN

In [10]:
query = \
'''
SELECT m.item_name, s.sale_date
FROM menu_table m
FULL OUTER JOIN sales_table s ON m.item_id = s.item_id;
'''
query_to_result(query)

,item_name,sale_date
0,Linear Regresso Latte,2023-10-01
1,Linear Regresso Latte,2023-10-03
2,Croissant,2023-10-01
3,Smoothie,2023-10-02
4,None,2023-10-04
5,Outlier Omelette,None
6,None,2023-10-05


### CROSS JOIN

A cross join in SQL produces the Cartesian product of the two involved tables, returning all possible combinations of rows from both tables, where each row from the first table is paired with each row from the second table.

Notice:
- Duplicate values: Cross joins are not duplicate insensitive.
- Missing values: Cross joins are not missing value insensitive.

If you have duplicate and/or missing values in your tables, you will get duplicate and/or missing values in your cross join.

In [11]:
query = \
'''
SELECT m.item_name, s.sale_date
FROM menu_table m
CROSS JOIN sales_table s;
'''
query_to_result(query)

,item_name,sale_date
0,Linear Regresso Latte,2023-10-01
1,Linear Regresso Latte,2023-10-01
2,Linear Regresso Latte,2023-10-02
3,Linear Regresso Latte,2023-10-03
4,Linear Regresso Latte,2023-10-04
5,Linear Regresso Latte,2023-10-05
6,Croissant,2023-10-01
7,Croissant,2023-10-01
8,Croissant,2023-10-02
9,Croissant,2023-10-03


We can handle missing and duplicate values in cross joins as follows:

In [12]:
query = \
'''
SELECT m.item_name, s.sale_date
FROM (SELECT DISTINCT item_name FROM menu_table WHERE item_name IS NOT NULL) m
CROSS JOIN (SELECT DISTINCT sale_date FROM sales_table WHERE sale_date IS NOT NULL) s;
'''
query_to_result(query)

,item_name,sale_date
0,Linear Regresso Latte,2023-10-01
1,Linear Regresso Latte,2023-10-02
2,Linear Regresso Latte,2023-10-03
3,Linear Regresso Latte,2023-10-04
4,Linear Regresso Latte,2023-10-05
5,Croissant,2023-10-01
6,Croissant,2023-10-02
7,Croissant,2023-10-03
8,Croissant,2023-10-04
9,Croissant,2023-10-05


A useful application for using a cross join:

In [13]:
query = \
'''
SELECT m.item_name, s.sale_date, 
       (SELECT COUNT(*) FROM sales_table st 
        JOIN menu_table mt ON st.item_id = mt.item_id
        WHERE mt.item_name = m.item_name AND st.sale_date = s.sale_date) as sale_count
FROM (SELECT DISTINCT item_name FROM menu_table WHERE item_name IS NOT NULL) m
CROSS JOIN (SELECT DISTINCT sale_date FROM sales_table WHERE sale_date IS NOT NULL) s;
'''
query_to_result(query)

,item_name,sale_date,sale_count
0,Linear Regresso Latte,2023-10-01,1
1,Linear Regresso Latte,2023-10-02,0
2,Linear Regresso Latte,2023-10-03,1
3,Linear Regresso Latte,2023-10-04,0
4,Linear Regresso Latte,2023-10-05,0
5,Croissant,2023-10-01,1
6,Croissant,2023-10-02,0
7,Croissant,2023-10-03,0
8,Croissant,2023-10-04,0
9,Croissant,2023-10-05,0


### SELF JOIN

A self join is a regular join operation in SQL where a table is joined with itself to relate rows within the same table.

A self join is typically used for hierarchical or organizational data where relationships within the same dataset need to be analyzed, such as finding the managerial chain in an employee database or arranging hierarchical categories where each category may have a parent category within the same table.

In [14]:
query = \
'''
SELECT *
FROM employees_table
'''
query_to_result(query)

,employee_id,employee_name,manager_id
0,1,Jiaxuan,5.0
1,2,Ireri,5.0
2,3,Rithvik,5.0
3,4,Bassim,5.0
4,5,Robert,NaN


In [15]:
query = \
'''
SELECT e1.employee_name, e2.employee_name AS manager_name
FROM employees_table e1
LEFT JOIN employees_table e2 ON e1.manager_id = e2.employee_id;
'''
query_to_result(query)

,employee_name,manager_name
0,Jiaxuan,Robert
1,Ireri,Robert
2,Rithvik,Robert
3,Bassim,Robert
4,Robert,None


Edge cases:
- Non-unique joining key: When using a non-unique joining key in a self join, it can result in a multiplication effect, producing more rows in the output than expected.
- Duplicate values: Duplicate values in the columns used for the self join can lead to redundancy and repetition in the output, making the results harder to interpret and analyze.

## SUMMARY OF JOINS

<div style="text-align:center">
    <img src="imgs/summary_plot.png" alt="alt text" title="Summary of Joins" width="600"/>
</div>